# import libraries

In [1]:
import os
import random

import pyvista as pv
import trimesh as tm
import numpy as np
import topogenesis as tg

from trimesh import voxel as v

In [2]:
# base 'rubik's cube'
unit = 45
division = 3
macrovoxel = int(unit/division)
base = tm.creation.box((unit,unit,unit))
newshape = base.apply_translation([60, -60, 0])

# import the environment

In [3]:
# load context file. Currently set as Rotterdam area from spatial computing
context_path = os.path.relpath('../data/immediate_context.obj')
context = tm.load(context_path)
context.apply_translation([0, 0, -unit/2])

<trimesh.Trimesh(vertices.shape=(8929, 3), faces.shape=(18283, 3))>

# create macro voxels

In [4]:
# manually create voxels
# voxel = newshape.voxelized(macrovoxel)
# voxel = voxel.as_boxes()

In [5]:
# voxelize the cube
base_lattice = tg.lattice(newshape.bounds, unit=[macrovoxel,macrovoxel,macrovoxel], default_value=1, dtype=int)

In [6]:
# testing configuration:

# create random configuration
config_rand = [random.randint(0,1) for i in range(27)]

# voxels of the configuration
# my_config = [vox_macro[i] for i in range(26) if config_rand[i] == 1]
config_index = [i for i in range(26) if config_rand[i] == 0]

config_rand = np.reshape(config_rand,(3,3,3))
config_rand = np.pad(config_rand, 1)

In [23]:
# create a step one moore neighbourhood
s_vn = tg.create_stencil("von_neumann", 1)

# set the center to 0
# s_vn.set_index([1, 0, 0], 1)
s_vn.set_index([0, 1, 0], 0)

# assign the sum function
s_vn.function = tg.sfunc.sum  # np.sum

s_vn

stencil([[[0, 0, 0],
          [0, 1, 0],
          [0, 0, 0]],

         [[0, 1, 0],
          [1, 1, 1],
          [0, 0, 0]],

         [[0, 0, 0],
          [0, 1, 0],
          [0, 0, 0]]])

In [56]:

below_stencil = tg.create_stencil("von_neumann", 2)*0

below_stencil.set_index([0, 0, 0], 1)
below_stencil.set_index([1, 0, 0], 1)
below_stencil.set_index([2, 0, 0], 1)

below_stencil.function = tg.sfunc.sum
below_stencil

nextto_stencil = tg.create_stencil("von_neumann", 2)*0

nextto_stencil.set_index([0,0,0],1)
nextto_stencil.set_index([0,0,-1],1)
nextto_stencil.set_index([0,0,-2],1)
nextto_stencil.function = tg.sfunc.sum
nextto_stencil

stencil([[[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]],

         [[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]],

         [[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [1, 1, 1, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]],

         [[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]],

         [[0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0]]])

In [57]:
config_lattice = base_lattice * config_rand
# config_lattice[-1] = np.ones((5,5))


In [60]:
# applying the mask to check if the voxel is not floating

# config_shifted_up = np.roll(config_lattice, (0,0,1), (0,1,2))
# foundation_mask = (config_lattice == True)*(config_shifted_up==False)
# foundation_mask_lattice = tg.to_lattice(foundation_mask, config_lattice)

aaaa = config_lattice.apply_stencil(nextto_stencil)
foundation_mask = aaaa * config_lattice >= 3
foundation_mask_lattice  = tg.to_lattice(foundation_mask, config_lattice)

# visualizing


In [61]:
# convert to trimesh definition, taken from spatial computing workshop
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# Visualize the mesh using pyvista plotter

# initiating the plotter
p = pv.Plotter(notebook=True)

# adding the base mesh: light blue
p.add_mesh(tri_to_pv(context), color='#abd8ff')
p.add_mesh(tri_to_pv(base), color='#abd8ff', opacity=0.2)
#p.add_mesh(tri_to_pv(newboxes), color='#804080', opacity=0.1)
#p.add_mesh(tri_to_pv(voxel), color='#804080', opacity=0.8)
foundation_mask_lattice.fast_vis(p)
# config_lattice.fast_vis(p)

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(788.6785661399646, 711.2621611399645, 723.5957336399646),
 (65.08283250000001, -12.333572500000002, 0.0),
 (0.0, 0.0, 1.0)]